In [1]:
# !pip install snowflake-connector-python

In [2]:
import snowflake.connector
import pandas as pd
import json

'''user = REFRACT.FOSFOR@LNTINFOTECH.COM
password = Password321
account = fya62509.us-east-1
warehouse = FOSFOR_REFRACT
database = FOSFOR_REFRACT
schema = SALES
role = FOSFOR_REFRACT'''    

# Snowflake connection parameters
snowflake_account = 'rwb76633'
snowflake_user = 'Ravi'
snowflake_password = 'Kayakave@1983'
snowflake_database = 'TUTORIAL_DB'
snowflake_schema = 'DATA_SCHEMA'
snowflake_stage = 'MAGICODER2'
role = 'TEST_ROLE'

# snowflake_account = 'fya62509.us-east-1'
# snowflake_user = 'REFRACT.FOSFOR@LNTINFOTECH.COM'
# snowflake_password = 'Password321'
# snowflake_database = 'FOSFOR_REFRACT'
# snowflake_schema = 'SALES'
# snowflake_stage = 'MAGICODER'
 
# Connect to Snowflake
conn = snowflake.connector.connect(
   user=snowflake_user,
   password=snowflake_password,
   account=snowflake_account,
   session_parameters={
      'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
   },
   database=snowflake_database,
   schema=snowflake_schema,
   role=role,
)




In [3]:
conn

In [3]:
#! ls -lrt /llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66

In [4]:
# Create a cursor object
cur = conn.cursor()
 
# Upload file to stage (example using Pandas)
#file_path = '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66/special_tokens_map.json'
# df = pd.read_csv(file_path)
#cur.execute(f"PUT file://{file_path} @{snowflake_stage}/magicoder_model AUTO_COMPRESS = FALSE")
#cur.execute("Select * from Sales")

In [165]:
prompt="""Generate documentation for following dbt code  {{ config(materialized='table') }}

with source as (
    select * from {{ source('src_table', 'employees') }}
),

parsed as (
    select
        id,
        PARSE_JSON(json_column):field1::varchar as field1,
        PARSE_JSON(json_column):field2::int as field2,
        PARSE_JSON(json_column):field3::timestamp as field3
    from source
)

select * from parsed"""


In [127]:
prompt = r"""Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios."""

In [166]:
prompt = prompt.replace("'", "''")
prompt = prompt.replace("\n", " ")

In [167]:
prompt

"Generate documentation for following dbt code  {{ config(materialized=''table'') }}  with source as (     select * from {{ source(''src_table'', ''employees'') }} ),  parsed as (     select         id,         PARSE_JSON(json_column):field1::varchar as field1,         PARSE_JSON(json_column):field2::int as field2,         PARSE_JSON(json_column):field3::timestamp as field3     from source )  select * from parsed"

In [101]:
prompt = "Generate bdt code to join supply and demand column"

In [86]:
temp_arg ={"prompt":prompt, "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}

In [87]:
temp_arg

{'prompt': "Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.",
 'max_tokens': 2000,
 'temperature': 0,
 'model': '/data/magicoder_model'}

In [113]:
type(arg)

str

In [163]:
arg = '{"prompt":'+'"'+prompt+'"'+', "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'
warehouse = 'USE WAREHOUSE TUTORIAL_WAREHOUSE;'
query = f"SELECT magicoder_udf(parse_json('{arg}'));"
print(query)

SELECT magicoder_udf(parse_json('{"prompt":"optimize below dbt code  {{ config(materialized=''table'') }}  with source as (     select * from {{ source(''src_table'', ''employees'') }} ),  parsed as (     select         id,         PARSE_JSON(json_column):field1::varchar as field1,         PARSE_JSON(json_column):field2::int as field2,         PARSE_JSON(json_column):field3::timestamp as field3     from source )  select * from parsed", "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'));


In [164]:
cur.execute(warehouse)
cur.execute(query)
result = cur.fetchall()
print(json.loads(result[0][0])['choices'][0]['text'])



```

The above dbt code is optimized as below:

```sql
with source as (
    select * from {{ source('src_table', 'employees') }}
),
parsed as (
    select
        id,
        json_extract_scalar(json_column, '$.field1')::varchar as field1,
        json_extract_scalar(json_column, '$.field2')::int as field2,
        json_extract_scalar(json_column, '$.field3')::timestamp as field3
    from source
)
select * from parsed
```

In the optimized version, we use the `json_extract_scalar` function to extract the values from the JSON column. This function is more efficient than the `PARSE_JSON` function and is supported by all major databases.


In [158]:
result

[('{\n  "choices": [\n    {\n      "finish_reason": "stop",\n      "index": 0,\n      "logprobs": null,\n      "text": ""\n    }\n  ],\n  "created": 900528,\n  "id": "cmpl-691f59d7581040858ef60c101927dfdd",\n  "model": "/data/magicoder_model",\n  "object": "text_completion",\n  "usage": {\n    "completion_tokens": 1,\n    "prompt_tokens": 74,\n    "total_tokens": 75\n  }\n}',)]

In [7]:
import requests
import json

# Obtain a session token.
token_data = conn._rest._token_request('ISSUE')
token_extract = token_data['data']['sessionToken']

# Create a request to the ingress endpoint with authz.
token = f'\"{token_extract}\"'
# Set this to the ingress endpoint URL for your service
url = 'http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions'

In [8]:
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}
print(headers)
print(url)
#print(payload)

{'Content-Type': 'application/json', 'Authorization': 'Snowflake Token="ver:1-hint:14544919633592326-ETMsDgAAAY5Vep50ABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEMk0A8Nx2TZnVqLTbZSljBcAAACA48T2Lr8JKf/dNoQ+63nJH9+QxeHcASiPik2NSxujbZIIjM54+IsUlHcpGUPDwSwQ1JRZTrKIgY0Yuui3WXfMCELevmH/ouvrrg2R/v+F9ovWKUZG0EcY9IHo/Wvqgd1AEOopoYMmfVV+nCaf6UMf9zYA8pcF9Bp2QhsVDUBPBOEAFLlv+fQcuY44iMx5FEtWCNf9RUCD"'}
http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions


In [9]:
payload = json.dumps({
  "prompt": "write a python code to check whether given string is pallindrome",
  "max_tokens": 2400,
  "temperature": 0,
  "model": "/data/magicoder_model"
})
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"detail":"Method Not Allowed"}
